In [ ]:
import pandas as pd

matchup_code = 'R6CH' # championship matchup
year = 2024

bracket_sims = pd.read_csv('./bracket_simulations.csv')

m_bracket_sims = bracket_sims[bracket_sims['Tournament'] == 'M']
# w_bracket_sims = bracket_sims[bracket_sims['Tournament'] == 'W']
    
seeds_m = pd.read_csv("march-machine-learning-mania-2025/MNCAATourneySeeds.csv")

# Have to fix X16, Y10, Y16, Z10 (change for 2025 seeds when we get them)
# X16 = Wagner = 1447
seeds_m = pd.concat([seeds_m, pd.DataFrame([{'Season': 2024, 'Seed': 'X16', 'TeamID': 1447}])], ignore_index=True)
# Y10 = Colorado St = 1161
seeds_m = pd.concat([seeds_m, pd.DataFrame([{'Season': 2024, 'Seed': 'Y10', 'TeamID': 1161}])], ignore_index=True)
# Y16 = Grambling = 1212
seeds_m = pd.concat([seeds_m, pd.DataFrame([{'Season': 2024, 'Seed': 'Y16', 'TeamID': 1212}])], ignore_index=True)
# Z10 = Colorado = 1160
seeds_m = pd.concat([seeds_m, pd.DataFrame([{'Season': 2024, 'Seed': 'Z10', 'TeamID': 1160}])], ignore_index=True)

teams = pd.read_csv("march-machine-learning-mania-2025/MTeams.csv")
seeds_m = seeds_m.merge(teams[['TeamID', 'TeamName']], on='TeamID')
seeds_m = seeds_m[seeds_m['Season'] == year]

seeds_m.head()

# Determining how many brackets were generated
num_brackets = bracket_sims['Bracket'].max()

# Labeling teams with the percentages of winning a particular matchup, given how many times they won in the simulation
value_counts = m_bracket_sims[m_bracket_sims['Slot'] == matchup_code]['Team'].value_counts()
percentages = (value_counts / num_brackets) * 100
team_names = seeds_m.set_index('Seed')['TeamName']
percentages.index = percentages.index.map(team_names)

# Represents the conditional probability that a team wins a particular matchup
percentages


Team
Connecticut       19.935
Houston           14.865
Purdue            11.857
Arizona            8.414
Iowa St            7.230
North Carolina     5.951
Tennessee          4.001
Marquette          2.768
Creighton          2.696
Illinois           2.619
Duke               2.614
Gonzaga            2.447
Baylor             1.972
St Mary's CA       1.959
Auburn             1.504
Kentucky           1.422
Wisconsin          0.845
South Carolina     0.714
Alabama            0.613
Clemson            0.575
San Diego St       0.570
Kansas             0.520
BYU                0.519
Texas              0.413
Washington St      0.372
Texas Tech         0.316
Michigan St        0.283
Florida            0.249
Mississippi St     0.180
Dayton             0.174
NC State           0.165
Northwestern       0.141
Oregon             0.119
Texas A&M          0.119
Nebraska           0.118
TCU                0.116
Utah St            0.101
Colorado           0.083
McNeese St         0.082
FL Atlantic        0

In [ ]:
# ...existing code...

def count_perfect_brackets(team_id, slot):
    """
    Count the number of perfect brackets remaining for a given team ID and slot.
    
    Parameters:
    team_id (int): The ID of the team.
    slot (str): The slot corresponding to a particular matchup.
    
    Returns:
    int: The number of perfect brackets remaining.
    """
    # Filter the bracket simulations for the given slot and team ID
    matching_brackets = m_bracket_sims[(m_bracket_sims['Slot'] == slot) & (m_bracket_sims['Team'] == team_id)]
    
    # Count the number of unique brackets that match the given team ID and slot
    perfect_brackets_count = matching_brackets['Bracket'].nunique()
    
    return perfect_brackets_count

# Example usage:
team_id = 1160  # Replace with the desired team ID
slot = 'R6CH'  # Replace with the desired slot
perfect_brackets = count_perfect_brackets(team_id, slot)
print(f"Number of perfect brackets remaining for team ID {team_id} in slot {slot}: {perfect_brackets}")